In [98]:
import pandas as pd

import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams    
import scipy.sparse as sps
import re
from unicodedata import normalize
from nltk.corpus import stopwords

In [99]:
news = pd.read_csv('date/estadao_noticias_eleicao.csv')

In [100]:
def limpar_texto(texto):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', texto)

In [101]:
words = [limpar_texto(stopword) for stopword in stopwords.words('portuguese')]

In [102]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")
content = content.apply(limpar_texto)

In [103]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [104]:
tokens_lists = content.apply(lambda text: text.lower().split())

In [105]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list if token not in words]

In [106]:
matrix, vocab = co_occurrence_matrix(tokens)

In [107]:
consultable_matrix = matrix.tocsr()

In [108]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [109]:
def get_co_ocurrence(word):
    list_of_occurency = consultable_matrix[vocab[word]].getrow(0).toarray()[0]
    indexs, frequency = zip(*sorted(enumerate(list_of_occurency), key=lambda x: x[1], reverse=True))
    return indexs[:3], frequency[:3]

ocurrecy = get_co_ocurrence('petrobras')
print([word for key in ocurrecy[0] for word in vocab.keys() if vocab[word] == key])

['paulo', 'graca', 'disse']


In [110]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)

3